In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 5.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.4 MB/s eta 0:00:0000:0100:01
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)


In [4]:
import pandas as pd 

train_org = pd.read_csv('train.csv') 
test_org = pd.read_csv('test.csv')

In [5]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다 
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 

# 해당 과정을 test_x에 대해서도 반복해줍니다 
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

In [6]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

In [7]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

In [8]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [10]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_x[i]) 
    train_x[i]=le.transform(train_x[i])
    
    for case in np.unique(test_x[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test_x[i]=le.transform(test_x[i])
    
display(train_x.head())
display(test_x.head())

['기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,6,2,0,0,2019,1,1,0,0,7,40,2,5
1,6,5,0,0,2019,1,1,0,0,1,4,2,5
2,6,2,0,0,2019,1,1,1,0,6,66,2,5
3,6,2,0,1,2019,1,1,2,0,4,79,2,5
4,6,2,0,1,2019,1,1,4,0,3,129,2,5


,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,토요일,2,0,0,2022,1,1,1,0,6,99,0,3
1,토요일,2,0,0,2022,1,1,1,0,6,168,2,5
2,토요일,2,0,1,2022,1,1,4,0,6,117,0,3
3,토요일,2,0,1,2022,1,1,4,0,6,126,2,5
4,토요일,2,0,1,2022,1,1,6,0,1,4,0,3


In [12]:

from autogluon.tabular import TabularDataset, TabularPredictor

# autogluon 학습을 위한 데이터 형태로 변환
train = TabularDataset(train_df.drop(['ID'], axis=1))
test = TabularDataset(test_df.drop(['ID'], axis=1))

/Users/itaeyun/opt/anaconda3/envs/dev/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


학습에
train_x 넣고 다시 ㄱㄱ

In [13]:
# 이렇게 한 줄만 작성하면 내부에서 알아서 학습해줍니다.
predictor = TabularPredictor(label='ECLO', eval_metric='f1_macro',).fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20231115_053715/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231115_053715/"
AutoGluon Version:  0.8.2
Python Version:     3.8.15
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.5.0: Mon Apr 24 20:53:44 PDT 2023; root:xnu-8796.121.2~5/RELEASE_ARM64_T8103
Disk Space Avail:   18.32 GB / 245.11 GB (7.5%)
Train Data Rows:    39609
Train Data Columns: 27
Label Column: ECLO
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 46) unique label values:  [5, 3, 6, 18, 9, 8, 15, 2, 1, 7]
	If 'multiclass' is not the correc

In [14]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
ld_board = predictor.leaderboard(train, silent=True)

ld_board

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.546259,0.928166,2.818266,0.410121,100632.330664,0.026470,0.003077,1.727369,2,True,7
1,XGBoost,0.542676,0.916385,0.759995,0.060848,26.018679,0.759995,0.060848,26.018679,1,True,5
2,CatBoost,0.539485,0.916340,1.504986,0.210783,100561.167082,1.504986,0.210783,100561.167082,1,True,4
3,KNeighborsDist,0.539269,0.195606,0.512194,0.057110,0.078938,0.512194,0.057110,0.078938,1,True,2
4,NeuralNetFastAI,0.529371,0.902909,1.075860,0.084150,104.187243,1.075860,0.084150,104.187243,1,True,3
5,NeuralNetTorch,0.519337,0.911745,0.922153,0.060212,205.693107,0.922153,0.060212,205.693107,1,True,6
6,KNeighborsUnif,0.146049,0.165057,0.526816,0.135414,43.417533,0.526816,0.135414,43.417533,1,True,1


In [20]:
test.head
test.tail

<bound method NDFrame.tail of         요일 기상상태 노면상태  사고유형     연   월   일  시간     도시    구      동 도로형태1  도로형태2
0      토요일   맑음   건조  차대사람  2022   1   1   1  대구광역시  수성구     상동   교차로   교차로안
1      토요일   맑음   건조  차대사람  2022   1   1   1  대구광역시  수성구    지산동   단일로     기타
2      토요일   맑음   건조   차대차  2022   1   1   4  대구광역시  수성구  수성동2가   교차로   교차로안
3      토요일   맑음   건조   차대차  2022   1   1   4  대구광역시  수성구    신매동   단일로     기타
4      토요일   맑음   건조   차대차  2022   1   1   6  대구광역시  달서구    감삼동   교차로   교차로안
...    ...  ...  ...   ...   ...  ..  ..  ..    ...  ...    ...   ...    ...
10958  토요일   맑음   건조   차대차  2022  12  31  18  대구광역시   남구    대명동   단일로     터널
10959  토요일   맑음   건조   차대차  2022  12  31  18  대구광역시  수성구    시지동   단일로     기타
10960  토요일   맑음   건조   차대차  2022  12  31  20  대구광역시  수성구    연호동   단일로     기타
10961  토요일   맑음   건조   차대차  2022  12  31  20  대구광역시  수성구    범물동   교차로  교차로부근
10962  토요일   맑음   건조   차대차  2022  12  31  21  대구광역시   동구    효목동   교차로  교차로부근

[10963 rows x 13 columns]>

In [21]:
print(train_df.columns)
print(test_df.columns)

Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',
       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',
       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',
       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],
      dtype='object')
Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',
       '도로형태1', '도로형태2'],
      dtype='object')


In [22]:
test = TabularDataset(test_df.drop(['ID'], axis=1))

# 예측하기
pred_y = predictor.predict(train)

In [23]:
sample_submission = pd.read_csv("sample_submission.csv")

baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = pred_y
baseline_submission 

,ID,ECLO
0,ACCIDENT_39609,5
1,ACCIDENT_39610,3
2,ACCIDENT_39611,3
3,ACCIDENT_39612,5
4,ACCIDENT_39613,3
...,...,...
10958,ACCIDENT_50567,5
10959,ACCIDENT_50568,11
10960,ACCIDENT_50569,3
10961,ACCIDENT_50570,3


In [24]:
baseline_submission.to_csv('first_submit.csv', index=False)